In [13]:
import pandas as pd
import re

# Specify the file path to the dataset
dataset_path = '../../data/final_database/auctions_main.csv'
# Read the dataset
data = pd.read_csv(dataset_path)


# Function to extract numeric value from a string
def extract_numeric_value(string):
    match = re.search(r'\d+(\.\d+)?', string)
    if match:
        return float(match.group())
    return None

# Create the new columns
data['noi_absolute_value'] = data['NOI'].apply(lambda x: extract_numeric_value(x) if isinstance(x, str) else None)
data['noi_absolute_value'] = data['noi_absolute_value']*1000000

# Round the 'noi_absolute_value' to 2 decimal places
data['noi_absolute_value'] = data['noi_absolute_value'].round(2)

data['noi_direction'] = data['NOI'].apply(lambda x: -1 if isinstance(x, str) and ('to buy' in x or 'bid' in x) else (1 if isinstance(x, str) and ('to sell' in x or 'offer' in x) else 0 if isinstance(x, str) and extract_numeric_value(x) == 0 else None))

data['currency'] = data['NOI'].apply(lambda x: re.findall(r'[^\d\s]+', x)[0] if isinstance(x, str) and re.findall(r'[^\d\s]+', x) else None)
# Print the updated DataFrame
print(data[['NOI', 'noi_absolute_value', 'noi_direction', 'currency']])




                              NOI  noi_absolute_value  noi_direction currency
0             $99 million to sell        9.900000e+07            1.0        $
1             $45 million to sell        4.500000e+07            1.0        $
2             $41 million to sell        4.100000e+07            1.0        $
3               $20m (offer side)        2.000000e+07            1.0        $
4             $77 million to sell        7.700000e+07            1.0        $
5        $30.5 million (bid side)        3.050000e+07           -1.0        $
6             $66 Million to sell        6.600000e+07            1.0        $
7             $60 million to sell        6.000000e+07            1.0        $
8              $12 million to buy        1.200000e+07           -1.0        $
9             $608 million to buy        6.080000e+08           -1.0        $
10             $79 million to buy        7.900000e+07           -1.0        $
11            $542 million to buy        5.420000e+08           

In [14]:
import os
import pandas as pd

database_path = '../../data/raw_auction_csv'

# Iterate over each row of the dataset
for index, row in data.iterrows():
    identifier = row['identifier']
    
    # Find the file based on the identifier
    matching_files = [file for file in os.listdir(database_path) if file.endswith('.csv') and "Final Results" in file and identifier in file]
    
    if matching_files:
        file_path = os.path.join(database_path, matching_files[0])
        df = pd.read_csv(file_path, header=None)
        
        # Check if the first line is "Relevant Currency"
        if df.iloc[0, 0] == "Relevant Currency":
            # Update the 'currency' column with the value from the second line
            data.loc[data['identifier'] == identifier, 'currency'] = df.iloc[1, 0]

# Print the updated DataFrame
print(data[['identifier', 'currency']])



                                     identifier currency
0                           20051104_delphi_CDS        $
1                          20060117_calpine_CDS        $
2                 20060331_dana_corporation_CDS        $
3                      20061128_dura_senior_CDS        $
4                20061128_dura_subordinated_CDS        $
5                   20071023_movie_gallery_LCDS        $
6                           20080219_quebcr_CDS        $
7                           20081002_tembec_CDS        $
8           20081006_famefrmc_Fannie-Mae-Senior        $
9     20081006_famefrmc_Fannie-Mae-Subordinated        $
10         20081006_famefrmc_Freddie-Mac-Senior        $
11   20081006_famefrmc_Freddie-Mac-Subordinated        $
12                          20081010_lehbro_CDS        $
13                          20081023_washmu_CDS        $
14                       20081104_landsb_Senior        €
15                 20081104_landsb_Subordinated        €
16                       200811

In [15]:
# Substitute $ with USD, € with EUR, and £ with GBP in the 'currency' column
data['currency'] = data['currency'].str.replace('$', 'USD').str.replace('€', 'EUR').str.replace('£', 'GBP')

# Remove all spaces from the 'currency' column
data['currency'] = data['currency'].str.replace(' ', '')

# Print the updated DataFrame
print(data[['identifier', 'currency']])



                                     identifier currency
0                           20051104_delphi_CDS      USD
1                          20060117_calpine_CDS      USD
2                 20060331_dana_corporation_CDS      USD
3                      20061128_dura_senior_CDS      USD
4                20061128_dura_subordinated_CDS      USD
5                   20071023_movie_gallery_LCDS      USD
6                           20080219_quebcr_CDS      USD
7                           20081002_tembec_CDS      USD
8           20081006_famefrmc_Fannie-Mae-Senior      USD
9     20081006_famefrmc_Fannie-Mae-Subordinated      USD
10         20081006_famefrmc_Freddie-Mac-Senior      USD
11   20081006_famefrmc_Freddie-Mac-Subordinated      USD
12                          20081010_lehbro_CDS      USD
13                          20081023_washmu_CDS      USD
14                       20081104_landsb_Senior      EUR
15                 20081104_landsb_Subordinated      EUR
16                       200811

/tmp/ipykernel_39105/1969070466.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['currency'] = data['currency'].str.replace('$', 'USD').str.replace('€', 'EUR').str.replace('£', 'GBP')


In [16]:
missing_currency_count = data['currency'].isnull().sum()
print("Number of missing values in 'currency' column:", missing_currency_count)


Number of missing values in 'currency' column: 1


In [17]:
import os
import pandas as pd

database_path = '../../data/raw_auction_csv'

# Iterate over each row of the dataset
for index, row in data.iterrows():
    identifier = row['identifier']
    currency = row['currency']
    exchange_rate = None
    currencies_exchange = None

    # Check if currency is USD or $
    if currency is not None and ('USD' in currency or currency == '$'):
        exchange_rate = 1
        currencies_exchange = "USD/USD"
        print(f"Identifier: {identifier} - Currency is USD or $, exchange rate set to 1")
    else:
        # Look for CSV files in the folder
        csv_files = [file for file in os.listdir(database_path) if file.endswith('.csv') and all(word not in file for word in ['Final Price', 'Limit Orders', 'NOI', 'IMM', 'Physical', 'Markets'])]
        #print(f"Identifier: {identifier} - CSV files in folder: {csv_files}")

        # Check if the identifier matches any CSV file name
        matching_files = [file for file in csv_files if identifier in file]
        print(f"Identifier: {identifier} - Matching files: {matching_files}")

        if matching_files:
            for csv_file in matching_files:
                print(f"Identifier: {identifier} - Selected CSV file: {csv_file}")
                csv_path = os.path.join(database_path, csv_file)
                df_csv = pd.read_csv(csv_path, header=None)
                print(f"Identifier: {identifier} - CSV file contents:\n{df_csv}")
                if 'Auction Currency Rates' in str(df_csv.iloc[0, 0]):
                    usd_rates = df_csv[df_csv.iloc[:, 0].str.contains('USD')]
                    if not usd_rates.empty:
                        exchange_rate = usd_rates.iloc[-1, 1]
                        currencies_exchange = usd_rates.iloc[-1, 0]
                        print(f"Identifier: {identifier} - Found 'Auction Currency Rates' in row: {currencies_exchange}")
                        print(f"Identifier: {identifier} - Exchange rate found: {exchange_rate}")
                        break  # Break the loop if a match is found
                    else:
                        exchange_rate = df_csv.iloc[-1, 1]
                        currencies_exchange = df_csv.iloc[-1, 0]
                        print(f"Identifier: {identifier} - Found 'Auction Currency Rates' in the last row")
                        print(f"Identifier: {identifier} - Exchange rate found: {exchange_rate}")


    # Update the exchange_rate column in the data at the corresponding row of the identifier
    data.loc[index, 'exchange_rate'] = exchange_rate
    data.loc[index, 'currencies_exchange'] = currencies_exchange

# Fill empty 'currency' values with the first three letters of 'currencies_exchange'
data['currency'].fillna(data['currencies_exchange'].str[:3], inplace=True)

# Fill values that have 'million', 'Million', or '.' with the first three letters of 'currencies_exchange'
data.loc[data['currency'].isin(['million', 'Million', '.']), 'currency'] = data['currencies_exchange'].str[:3]


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Print the updated DataFrame
print(data[['identifier', 'NOI', 'noi_absolute_value','currency', 'exchange_rate', 'currencies_exchange']])


Identifier: 20051104_delphi_CDS - Currency is USD or $, exchange rate set to 1
Identifier: 20060117_calpine_CDS - Currency is USD or $, exchange rate set to 1
Identifier: 20060331_dana_corporation_CDS - Currency is USD or $, exchange rate set to 1
Identifier: 20061128_dura_senior_CDS - Currency is USD or $, exchange rate set to 1
Identifier: 20061128_dura_subordinated_CDS - Currency is USD or $, exchange rate set to 1
Identifier: 20071023_movie_gallery_LCDS - Currency is USD or $, exchange rate set to 1
Identifier: 20080219_quebcr_CDS - Currency is USD or $, exchange rate set to 1
Identifier: 20081002_tembec_CDS - Currency is USD or $, exchange rate set to 1
Identifier: 20081006_famefrmc_Fannie-Mae-Senior - Currency is USD or $, exchange rate set to 1
Identifier: 20081006_famefrmc_Fannie-Mae-Subordinated - Currency is USD or $, exchange rate set to 1
Identifier: 20081006_famefrmc_Freddie-Mac-Senior - Currency is USD or $, exchange rate set to 1
Identifier: 20081006_famefrmc_Freddie-Mac

In [18]:
# Replace the null values in currency_exchange with the combination of currency1/currency2
data['currencies_exchange'].fillna(data['currency'] + '/' + data['currency'], inplace=True)


In [19]:
# Set the exchange rate to 1 for the rows where currency_exchange is None
data.loc[data['exchange_rate'].isnull(), 'exchange_rate'] = 1

In [20]:
data[data['exchange_rate'].isnull()]

,identifier,auction_name,date,ticker,box_title,multiple_final_prices,final_price,IMM,NOI,noi_absolute_value,noi_direction,currency,exchange_rate,currencies_exchange


In [21]:
from currency_converter import CurrencyConverter
from dateutil import parser

# Create an instance of CurrencyConverter
c = CurrencyConverter()

# Function to get the exchange rate from USD to a specific currency on a given date
def get_exchange_rate(date, currency):
    return c.convert(1, 'USD', currency, date=date)

# Iterate over each row of the dataset
for index, row in data.iterrows():
    currency = row['currency']
    currencies_exchange = row['currencies_exchange']
    date_str = row['date']

    # Parse the date string into a datetime object
    date = parser.parse(date_str).date()

    # Check if the currency is not USD and 'USD' is not present in currencies_exchange
    if currency != 'USD': #and not re.search(r'USD', currencies_exchange)
        # Get the exchange rate from USD to the specific currency on the given date
        exchange_rate = get_exchange_rate(date, currency)
        currencies_exchange = 'USD/' + currency

        # Update the exchange_rate and currencies_exchange columns
        data.at[index, 'exchange_rate'] = exchange_rate
        data.at[index, 'currencies_exchange'] = currencies_exchange
    # Convert 'noi_absolute_value' to USD
    data.at[index, 'noi_usd'] = row['noi_absolute_value'] / exchange_rate

        
# Print the updated DataFrame
print(data)





                                     identifier  \
0                           20051104_delphi_CDS   
1                          20060117_calpine_CDS   
2                 20060331_dana_corporation_CDS   
3                      20061128_dura_senior_CDS   
4                20061128_dura_subordinated_CDS   
5                   20071023_movie_gallery_LCDS   
6                           20080219_quebcr_CDS   
7                           20081002_tembec_CDS   
8           20081006_famefrmc_Fannie-Mae-Senior   
9     20081006_famefrmc_Fannie-Mae-Subordinated   
10         20081006_famefrmc_Freddie-Mac-Senior   
11   20081006_famefrmc_Freddie-Mac-Subordinated   
12                          20081010_lehbro_CDS   
13                          20081023_washmu_CDS   
14                       20081104_landsb_Senior   
15                 20081104_landsb_Subordinated   
16                       20081105_glitni_Senior   
17                 20081105_glitni_Subordinated   
18                       200811

In [22]:
unique_currencies = data['currency'].unique()
print(unique_currencies)


['USD' 'EUR' 'GBP' 'JPY']


In [23]:
currency_values = [ '.']
mask = data['currency'].isin(currency_values)
identifiers = data.loc[mask, 'identifier'].tolist()
print(identifiers)


[]


In [24]:
# Specify the file path and name for saving the updated DataFrame
output_folder = "../../data/final_database"
os.makedirs(output_folder, exist_ok=True)
output_csv = os.path.join(output_folder, "auctions_main_updated.csv")

# Save the updated DataFrame as CSV
data.to_csv(output_csv, index=False)
print(f"Updated DataFrame saved as: {output_csv}")


Updated DataFrame saved as: ../../data/final_database/auctions_main_updated.csv
